# Section 3: Basic Bayesian Time Series Models

#### PyData London 2025 - Bayesian Time Series Analysis with PyMC

---

## Gaussian Random Walks

Random walks are fundamental building blocks for time series models. They're particularly useful for modeling:
- **Trends** that change smoothly over time
- **Level shifts** in time series
- **Latent states** in state-space models

### Mathematical Foundation

A Gaussian Random Walk is defined as:

$$x_t = x_{t-1} + \epsilon_t$$

where $\epsilon_t \sim \mathcal{N}(0, \sigma^2)$

With drift (trend):
$$x_t = x_{t-1} + \mu + \epsilon_t$$

### Why Random Walks?

- **Flexibility**: Can model various trend patterns
- **Smoothness**: Changes are gradual, not abrupt
- **Uncertainty**: Natural way to model evolving uncertainty
- **Interpretability**: Parameters have clear meanings

In [ ]:
# Import necessary libraries for Section 3
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
import warnings

# Configure plotting and suppress warnings
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
RANDOM_SEED = 42

print("🔧 Section 3 libraries loaded successfully!")
print("Ready to build basic Bayesian time series models")

In [ ]:
# Load and prepare data (consistent with previous sections)
births_data = pl.read_csv('../data/births.csv', null_values=['null', 'NA', '', 'NULL'])
births_data = births_data.filter(pl.col('day').is_not_null())

monthly_births = (births_data
    .group_by(['year', 'month'])
    .agg(pl.col('births').sum())
    .sort(['year', 'month'])
)

births_subset = (monthly_births
    .filter((pl.col('year') >= 1970) & (pl.col('year') <= 1990))
    .with_row_index('index')
)

# Standardize the data
original_data = births_subset['births'].to_numpy()
births_standardized = (original_data - original_data.mean()) / original_data.std()
n_obs = len(births_standardized)

print(f"📊 Data prepared: {n_obs} observations")
print(f"   Date range: {births_subset['year'].min()}-{births_subset['year'].max()}")

## Model 1: Simple Random Walk

Let's start with the simplest time series model: a random walk without drift. This model assumes that each observation is the previous observation plus some random noise.

In [ ]:
# Model 1: Simple Random Walk
with pm.Model() as random_walk_model:
    # Step size of the random walk
    sigma_walk = pm.HalfNormal('sigma_walk', sigma=1.0)
    
    # Initial value distribution
    init_dist = pm.Normal.dist(mu=0, sigma=1)
    
    # Gaussian random walk
    walk = pm.GaussianRandomWalk('walk', 
                                mu=0,  # no drift
                                sigma=sigma_walk, 
                                init_dist=init_dist,
                                steps=n_obs-1)
    
    # Observation noise
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=1.0)
    
    # Likelihood: observed data are noisy observations of the walk
    y_pred = pm.Normal('y_pred', mu=walk, sigma=sigma_obs, observed=births_standardized)

# Sample from the model
with random_walk_model:
    trace_rw = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("Random Walk Model Summary:")
print(az.summary(trace_rw, var_names=['sigma_walk', 'sigma_obs']))

## Model 2: Random Walk with Drift

Now let's add a drift term to capture any underlying trend in the data.

In [ ]:
# Model 2: Random Walk with Drift
with pm.Model() as rw_drift_model:
    # Drift parameter
    mu_drift = pm.Normal('mu_drift', mu=0, sigma=0.1)
    
    # Innovation variance
    sigma_walk = pm.HalfNormal('sigma_walk', sigma=0.5)
    
    # Random walk with drift
    rw_drift = pm.GaussianRandomWalk('rw_drift', 
                                    mu=mu_drift,  # Drift term
                                    sigma=sigma_walk,
                                    steps=n_obs-1)
    
    # Observation equation
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=0.5)
    obs = pm.Normal('obs', mu=rw_drift, sigma=sigma_obs, observed=births_standardized[1:])

# Sample from the model
with rw_drift_model:
    trace_rw_drift = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("Random Walk with Drift Model Summary:")
print(az.summary(trace_rw_drift, var_names=['mu_drift', 'sigma_walk', 'sigma_obs']))

## Autoregressive (AR) Models

Autoregressive models capture temporal dependence by relating current observations to past observations. They're fundamental to time series analysis.

### AR(1) Model

An AR(1) model is defined as:
$$y_t = \phi y_{t-1} + \epsilon_t$$

where $|\phi| < 1$ for stationarity and $\epsilon_t \sim \mathcal{N}(0, \sigma^2)$

In [ ]:
# Model 3: AR(1) Model
with pm.Model() as ar1_model:
    # AR coefficient (ensure stationarity by using a bounded prior)
    rho = pm.Beta('rho', alpha=1, beta=1)  # Beta(1,1) = Uniform(0,1) for stationarity
    # Transform to (-1, 1) for AR coefficient
    phi = pm.Deterministic('phi', 2 * rho - 1)
    
    # Innovation variance
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    # AR(1) process
    ar1 = pm.AR('ar1', rho=phi, sigma=sigma, constant=False, steps=n_obs-1)
    
    # Likelihood - observed data
    obs = pm.Normal('obs', mu=ar1, sigma=0.1, observed=births_standardized[1:])

# Sample from AR(1) model
with ar1_model:
    trace_ar1 = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("AR(1) Model Summary:")
print(az.summary(trace_ar1, var_names=['phi', 'sigma']))

## Model Comparison

Let's compare our three basic models to see which one fits the data best.

In [ ]:
# Compare models using WAIC (Widely Applicable Information Criterion)
models_dict = {
    'Random Walk': trace_rw,
    'RW with Drift': trace_rw_drift,
    'AR(1)': trace_ar1
}

# Compute WAIC for model comparison
comparison = az.compare(models_dict, ic='waic')
print("\n📊 Model Comparison (WAIC):")
print(comparison)

# Plot model comparison
az.plot_compare(comparison, figsize=(10, 6))
plt.title('Model Comparison: WAIC Scores')
plt.show()

print("\n💡 **Interpretation**:")
print("   • Lower WAIC indicates better model fit")
print("   • dWAIC shows difference from best model")
print("   • Weight shows relative model probability")

## Bayesian Regression with Trend and Seasonality

For time series with clear trend and seasonal patterns, we can build regression models that explicitly model these components.

In [ ]:
# Model 4: Bayesian Regression with Trend and Seasonality
# Create time index and seasonal components
time_idx = np.arange(n_obs)
time_normalized = (time_idx - time_idx.mean()) / time_idx.std()

# Create seasonal components (annual cycle)
period = 12  # Monthly data with annual seasonality
seasonal_freq = 2 * np.pi / period
sin_seasonal = np.sin(seasonal_freq * time_idx)
cos_seasonal = np.cos(seasonal_freq * time_idx)

with pm.Model() as seasonal_model:
    # Intercept
    mu_overall = pm.Normal('mu_overall', mu=0, sigma=1)
    
    # Trend coefficient
    beta_trend = pm.Normal('beta_trend', mu=0, sigma=1)
    
    # Seasonal coefficients
    beta_sin = pm.Normal('beta_sin', mu=0, sigma=1)
    beta_cos = pm.Normal('beta_cos', mu=0, sigma=1)
    
    # Expected value
    mu_t = pm.Deterministic('mu_t', 
                           mu_overall + 
                           beta_trend * time_normalized + 
                           beta_sin * sin_seasonal + 
                           beta_cos * cos_seasonal)
    
    # Observation noise
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=1)
    
    # Likelihood
    y_obs = pm.Normal('y_obs', mu=mu_t, sigma=sigma_obs, observed=births_standardized)

# Sample from seasonal model
with seasonal_model:
    trace_seasonal = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("Seasonal Model Summary:")
print(az.summary(trace_seasonal, var_names=['mu_overall', 'beta_trend', 'beta_sin', 'beta_cos', 'sigma_obs']))

## Summary

In this section, we've built and compared several basic Bayesian time series models:

1. **Random Walk**: Captures smooth trends without drift
2. **Random Walk with Drift**: Adds systematic trend component
3. **AR(1) Model**: Captures temporal dependence through autoregression
4. **Seasonal Regression**: Explicitly models trend and seasonality

### Key Insights

- **Model selection**: Use information criteria (WAIC, LOO) to compare models
- **Interpretability**: Bayesian models provide uncertainty for all parameters
- **Flexibility**: Can easily add or remove components based on data characteristics
- **Prior specification**: Informative priors can improve model performance

**Next**: In Section 4, we'll explore advanced models including state-space models, stochastic volatility, and Gaussian processes.

---

**Key Takeaways**:
- Start with simple models and add complexity gradually
- Random walks are excellent for modeling smooth trends
- AR models capture temporal dependence effectively
- Bayesian regression can explicitly model trend and seasonality
- Always compare models using appropriate criteria